# make Rscript

In [30]:
%%bash
workpath=/data2/xiangwei/projects/zhaolian/16cellchat/cellchatEpidownsample/slurm_scripts
cd $workpath
for REP in {1..50} 
do


echo "library(Seurat)
library(tidyverse)
library(CellChat)
library(patchwork)
library(repr)
library(RColorBrewer)
library(scales)
library(paletteer)
library(readxl)
options(stringsAsFactors = FALSE)

library(corrplot)
library(ggpubr)
options(future.globals.maxSize = 5e9)

setwd(\"/data2/xiangwei/projects/zhaolian/16cellchat/cellchatEpidownsample/ds$REP\")

# load data
scdata=readRDS(\"/data2/xiangwei/projects/zhaolian/16cellchat/cellchatEpidownsample/epidata\")
dsRes=read.table(\"/data2/xiangwei/projects/zhaolian/16cellchat/cellchatEpidownsample/downsampleRes.csv\",sep = \",\",header = T)
# select replicate
cellID=dsRes%>%filter(replicate==$REP)
cellID=cellID\$cellID
scdata=subset(scdata,cells=cellID)
sampleID=names(table(scdata\$sampleID))
sampleID
cellnum=table(scdata\$fine,scdata\$sampleID)%>%as.data.frame.array()
write.table(cellnum,\"results/cellnum.csv\",sep = \",\")
# get matrixdata
sampleMatrixList=vector(\"list\",length = length(sampleID))
sampleMetadataList=vector(\"list\",length = length(sampleID))
names(sampleMatrixList)=sampleID
names(sampleMetadataList)=sampleID
Idents(scdata)=\"sampleID\"
for (i in seq_along(sampleMatrixList)){
  sampleMatrix=subset(scdata,idents = sampleID[i])
  cutoff=0.01*ncol(sampleMatrix)
  cutoffCellType=rownames(table(sampleMatrix\$fine)%>%as.data.frame.array()%>% set_names(\"num\")%>% filter(num>!!cutoff))
  
  Idents(sampleMatrix)=\"fine\"
  selectedMatrix=subset(sampleMatrix,idents = cutoffCellType)
  scaledMatrix=selectedMatrix[[\"SCT\"]]\$data
  
  sampleMatrixList[[i]]=scaledMatrix
  sampleMetadataList[[i]]=selectedMatrix@meta.data
}
# Create a CellChat object
cellchatObj=vector(\"list\",length = length(sampleID))
names(cellchatObj)=sampleID
for (i in seq_along(cellchatObj)){
  cellchatObj[[i]]=createCellChat(object = sampleMatrixList[[i]], meta = sampleMetadataList[[i]], group.by = \"fine\")
}
for (i in seq_along(sampleID)){
  cellchatObj[[i]]@DB=CellChatDB.mouse
  cellchatObj[[i]] = subsetData(cellchatObj[[i]])
  cellchatObj[[i]] = identifyOverExpressedGenes(cellchatObj[[i]])
  cellchatObj[[i]] = identifyOverExpressedInteractions(cellchatObj[[i]])
  cellchatObj[[i]] = projectData(cellchatObj[[i]], PPI.mouse)
  cellchatObj[[i]] = computeCommunProb(cellchatObj[[i]],raw.use = FALSE,population.size = F)
  df.net = subsetCommunication(cellchatObj[[i]])
  df.netP = subsetCommunication(cellchatObj[[i]],slot.name = \"netP\")
  write.table(df.net,str_c(\"results/\",sampleID[i],\"dfNet.csv\"),sep = \",\",row.names = F)
  write.table(df.netP,str_c(\"results/\",sampleID[i],\"dfNetP.csv\"),sep = \",\",row.names = F)
  cellchatObj[[i]] = computeCommunProbPathway(cellchatObj[[i]])
  cellchatObj[[i]] = aggregateNet(cellchatObj[[i]])
  write.table(cellchatObj[[i]]@net[[\"count\"]],str_c(\"results/\",sampleID[i],\"count.csv\"),sep = \",\")
  write.table(cellchatObj[[i]]@net[[\"weight\"]],str_c(\"results/\",sampleID[i],\"weight.csv\"),sep = \",\")
}
saveRDS(cellchatObj,\"results/cellchatObjResPopF\")
#stat
totalLRnum=vector(\"numeric\",length = length(sampleID))
totalLRweight=vector(\"numeric\",length = length(sampleID))
names(totalLRnum)=sampleID
names(totalLRweight)=sampleID

for (i in seq_along(totalLRnum)){
  totalLRnum[i]=sum(cellchatObj[[i]]@net[[\"count\"]])
  totalLRweight[i]=sum(cellchatObj[[i]]@net[[\"weight\"]])
}

LRtable=tibble(sampleID=sampleID,totalLRnum=totalLRnum,totalLRweight=totalLRweight)

npvalue=read_xlsx(\"/data2/xiangwei/projects/zhaolian_old/BfinalSample/Np_new2.xlsx\")%>% select(sampleID,Np,\`1/Np\`) %>% 
  mutate(stage=if_else(Np>4,\"early\",\"late\"))%>%as.data.frame()
npvalue
#merge
NpLRtable=npvalue%>%left_join(LRtable,by=\"sampleID\")
write.table(NpLRtable,\"results/NpLRtable.csv\",sep = \",\",row.names = F)

plot=ggplot(NpLRtable,aes(x=\`1/Np\`,y=totalLRnum))+geom_point()+theme_classic()+
  geom_smooth(method = \"lm\", formula = y ~ x, color = \"black\", fill = \"#b2e7fa\",alpha = 0.4)+
  stat_cor(method = \"spearman\",label.x = 0,label.y = 0)+xlim(c(0,0.65))+
  ylab(c(\"Total L-R Paires numbers per sample\"))
ggsave(plot,filename = \"results/plot/corLRnp.pdf\",width = 3.5,height = 3)"> ds$REP.script

done

# make slurm 

In [17]:
%%bash
workpath=/data2/xiangwei/projects/zhaolian/16cellchat/cellchatEpidownsample/slurm_scripts
cd $workpath
for REP in {1..50} 
do

echo "sbatch -J REP$REP -p fat -N 1 -n 2 --mem=5G  -t 0 -o ./err${REP} -e ./out${REP} --wrap=\"/home/xiangwei/miniconda3/envs/seurat5/bin/Rscript $workpath/ds$REP.script\"" >> slurm.sh

done
chmod 777 slurm.sh

# merge data

In [31]:
%%bash
workpath=/data2/xiangwei/projects/zhaolian/16cellchat/cellchatEpidownsample
for REP in {1..50} 
do
cd $workpath/ds$REP/results

cp cellnum.csv $workpath/summary/cellNum/ds${REP}_cellnum.csv
cp NpLRtable.csv $workpath/summary/LRnum/ds${REP}_NpLRtable.csv
cp plot/corLRnp.pdf $workpath/summary/plot/ds${REP}_corLRnp.pdf
done